In [ ]:
!pip install pathlib
!pip install tokenizers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 7.6 MB 4.6 MB/s 


In [ ]:
!pip install konlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 19.4 MB 1.1 MB/s 
     |████████████████████████████████| 465 kB 48.1 MB/s 


In [ ]:
from konlpy.tag import Okt
import pandas as pd

from tqdm.notebook import tqdm

In [ ]:
okt_sentence = pd.read_csv('/content/drive/Shareddrives/SW2_0916/3ck/dataset/aihub_okt.csv')

In [ ]:
yt_comments = list()
with open('/content/drive/Shareddrives/SW2_0916/3ck/dataset/yt_commens.csv', 'r') as f:
  for line in f.readlines()[1:]:
    yt_comments.append(line)

In [ ]:
yt_cmt = pd.DataFrame(yt_comments, columns= ['text'])

In [ ]:
yt_cmt.text = yt_cmt.text.str.slice(0,-1)

In [ ]:
import ast
okt_sentence.okt_text = okt_sentence.okt_text.map(ast.literal_eval)

In [ ]:
okt_sentence.text

0                         부랴부랴 왔는데 아무도 안왔네. 시간개념들이 없네
1                                   맞아. 사람들이 진짜 개념이없다
2                                        저렇게는 살지 말아야지
3                              그러게 게으르고 멍청한 사람들은 맞아야해
4                                   인방 보는 남자는 거르는게 맞다
                             ...                     
408364                               저 여자는 오늘도 레깅스 입네
408365                           남자 아래에 깔려서 당하고 싶은건가?
408366                          휴유, 내가 봐준다 뒷태가 끝내주는데!
408367            저정도면 놀아달라는 거 아닌가? 저 라인봐봐 만지면 으스러지겠어
408368    저 언덕 사이에 손을 넣고 주물러주면 그럴걸 ㅋㅋ 본인도 바라고 있지 않을까?
Name: text, Length: 408369, dtype: object

In [ ]:
total = pd.concat([okt_sentence.text, yt_cmt.text])

In [ ]:
okt = Okt()
okt_total = list()
for text in tqdm(total):
  okt_total.append(' '.join(okt.morphs(text)).strip())

  0%|          | 0/505771 [00:00<?, ?it/s]

In [ ]:
okt_df = pd.DataFrame(okt_total, columns=['text'])

In [ ]:
okt_df.iloc[:20]

,text
0,부랴부랴 왔는데 아무 도 안 왔네 . 시간 개념 들 이 없네
1,맞아 . 사람 들 이 진짜 개념 이 없다
2,저렇게는 살 지 말아야지
3,그러게 게으르고 멍청한 사람 들 은 맞아야해
4,인방 보는 남자 는 거르는 게 맞다
5,특히 벗방 보는 애 들 은 진짜 거세 시켜 야함
6,볼 게 없어서 벗방 을 보냐 ㅋㅋㅋ
7,사회 성 이 얼마나 떨어지면 그러냐 ㅋㅋㅋ
8,댓글 에 빠순이 들 몰려와서 즈그 주인 님 쉴드치는 꼴 좀 봐
9,이래서 인 방충 ~ 인 방충 ~ 하는거 구나


In [ ]:
total.to_csv('/content/drive/Shareddrives/SW2_0916/3ck/dataset/total_text.txt', index=False, header=False)

In [ ]:
my_vocab_size = 32000
# vocab의 크기를 의미한다. 적을 수록 "단어" 단위로, 클 수록 "음절" 단위로 나뉘어진다.
my_limit_alphabet = 6000
# 모든 알파벳을 커버할 수 있도록 하여, [UNK] 빈도를 줄이기 위해 6000을 선택했다.
my_special_tokens = ["[PAD]", "[UNK]", "[CLS]", "[SEP]", "[MASK]"]
# tokenizer 에서 사용될 special_tokens이다. 필수 토큰은 위와 같다.
user_defined_symbols = ['[BOS]','[EOS]']
# 이제부터는 부가적인 토큰이다. 문장의 시작과 끝을 알리는 토큰을 추가했다.
unused_token_num = 200
unused_list = ['[unused{}]'.format(n) for n in range(unused_token_num)]
# KoELECTRA Github를 참고하여, unused 토큰을 약 200개 추가했다. 범용성을 높일 수 있다.
user_defined_symbols = user_defined_symbols + unused_list
my_special_tokens = my_special_tokens + user_defined_symbols

In [ ]:
from tokenizers import BertWordPieceTokenizer

tokenizer = BertWordPieceTokenizer(
    clean_text=True,
    handle_chinese_chars=True,
    strip_accents=False, 
    # 만약 cased model이라면 반드시 False로 해야한다, 또한 한글의 경우 cased model로 하면 글자가 자소분리된다.
    lowercase=False,
    # 대소문자 구분 여부를 의미한다. 한글의 경우 무의미하므로 신경쓰지 않아도 된다.
    wordpieces_prefix="##"
)

tokenizer.train(
    files='/content/drive/Shareddrives/SW2_0916/3ck/dataset/total_text.txt',
    limit_alphabet=my_limit_alphabet,
    vocab_size=my_vocab_size,
    min_frequency=5,
    # pair가 5회이상 등장할시에만 학습
    show_progress=True,
    # 진행과정 출력 여부
    special_tokens=my_special_tokens
)

In [ ]:
tokenizer.save("/content/drive/Shareddrives/SW2_0916/3ck/tokenizer/total/tok_added-ch-{}-wpm-{}".format(my_limit_alphabet, my_vocab_size),True)
# config, model등 저장하지 않고 vocab 정보만 json 형태로 저장

import json
vocab_path = "/content/drive/Shareddrives/SW2_0916/3ck/tokenizer/total/tok_added-ch-{}-wpm-{}".format(my_limit_alphabet, my_vocab_size)
# save의 결과로 추출된 파일 경로
vocab_file = '/content/drive/Shareddrives/SW2_0916/3ck/tokenizer/total/vocab.txt'
# vocab.txt 형태로 저장할 경로
f = open(vocab_file,'w',encoding='utf-8')
with open(vocab_path) as json_file:
    json_data = json.load(json_file)
    for item in json_data["model"]["vocab"].keys():
        f.write(item+'\n')

    f.close()

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from transformers import BertTokenizer
vocab_path = "/content/drive/Shareddrives/SW2_0916/3ck/tokenizer/total/vocab.txt"
tokenizer = BertTokenizer(vocab_file=vocab_path, do_lower_case=True) 
tokenizer.save_pretrained('/content/drive/Shareddrives/SW2_0916/3ck/tokenizer/total/')

('/content/drive/Shareddrives/SW2_0916/3ck/tokenizer/total/tokenizer_config.json',
 '/content/drive/Shareddrives/SW2_0916/3ck/tokenizer/total/special_tokens_map.json',
 '/content/drive/Shareddrives/SW2_0916/3ck/tokenizer/total/vocab.txt',
 '/content/drive/Shareddrives/SW2_0916/3ck/tokenizer/total/added_tokens.json')